<a href="https://colab.research.google.com/github/czarodziejszyn/ssne/blob/main/projekt2/predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import torch.nn as nn
import pandas as pd
import torch.utils.data as data
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
class MyNetBN(nn.Module):
    def __init__(self):
        super(MyNetBN, self).__init__()
        self.lin1 =nn.Linear(9, 48)  # 28 x 28 = 784
        self.bn1 = nn.BatchNorm1d(48)
        self.act1 =nn.ReLU()
        self.lin2 =nn.Linear(48, 24)
        self.bn2 = nn.BatchNorm1d(24)
        self.act2 =nn.ReLU()
        self.lin3 =nn.Linear(24, 1)


    def forward(self, x):
        x = self.lin1(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.bn2(x)
        x = self.act2(x)
        x = self.lin3(x)
        return x

In [6]:
train_data = pd.read_csv('train_data.csv')

In [9]:
num_cols = ['SalePrice', 'YearBuilt', 'Size(sqf)', 'Floor',
            'N_Parkinglot(Ground)', 'N_Parkinglot(Basement)', 'N_manager',
            'N_elevators', 'N_FacilitiesNearBy(Total)', 'N_SchoolNearBy(Total)']
cat_cols = ['HallwayType', 'HeatingType', 'AptManageType', 'TimeToBusStop',
            'TimeToSubway', 'SubwayStation']

ohe_cols = ['HallwayType', 'HeatingType', 'AptManageType', 'SubwayStation']
df = pd.get_dummies(train_data, columns=ohe_cols)

le = LabelEncoder()
df['TimeToBusStop'] = le.fit_transform(df['TimeToBusStop'])
df['TimeToSubway'] = le.fit_transform(df['TimeToSubway'])

In [ ]:
test_data = pd.read_csv('test_data.csv')